# Chromosome Encoding

In [6]:
'''
I think I'll represent my chromosomes as follows:
    chromosome = np.array([(int,int)]) where int >= 0
    
    for example:
    inital_sol = np.array([(0,0), (1,1), (4,4), (8,8)])

    the chromosomes should also not include the start and endpoint because that will never change.

    np.array encapsulates it in numpy which makes it easier to work with since the software is 
    wrapped in C for optimal performance.


'''

"\nI think I'll represent my chromosomes as follows:\n    chromosome = np.array([(int,int)]) where int >= 0\n    \n    for example:\n    inital_sol = np.array([(0,0), (1,1), (4,4), (8,8)])\n\n    the chromosomes should also not include the start and endpoint because that will never change.\n\n    np.array encapsulates it in numpy which makes it easier to work with since the software is \n    wrapped in C for optimal performance.\n\n\n"

# Roulette Selection Operator

In [26]:
'''
    creating functions for roulette selection
'''
import random
import numpy as np
def example_fitness(x):
    return random.randint(0,5)
def example_population(n: int):
    return np.array([i for i in range(10)])
def example_calc_fitness_prob(population, fitness_function):
    fitness_pop = np.array([fitness_function(i) for i in population])
    sum_fitness = np.sum(fitness_pop)
    return (fitness_pop / sum_fitness) #probability list
def roulette_selection(population, p_weights, n=2):
    #n is number to choose from
    return np.random.choice(population, 2, p=p_weights, replace=False)

In [27]:
population = example_population(10)
p_weights = example_calc_fitness_prob(population, example_fitness)
selected = roulette_selection(population, p_weights)

In [28]:
selected

array([4, 8])

# Single-Point Crossover

Description: Single point crossover takes the longest length gene and uses the index <br>
in the middle of path to split both of the paths being crossed over<br>

In [63]:
def crossover(parents: [[(int,int)]]):
    '''
        parents is a 2d matrix with each row representing a parent path
                    each column represents nth point of the path.
                    
        offspring_size is a 2d matrix too that represents the same thing
    '''
    #initial checks
    assert parents.shape[0] >= 2
    assert parents.shape[0] % 2 == 0
    assert len(parents[0]) != 0
    
    offspring = []
    for i in range(0,parents.shape[0],2): #iterate through each parent by 2
        parent1_idx,parent2_idx = i,i+1
        crossover_point = max(len(parents[parent1_idx]),len(parents[parent2_idx])) // 2
        offspring_path1 = parents[parent1_idx][0:crossover_point] + parents[parent2_idx][crossover_point:]
        offspring_path2 = parents[parent2_idx][0:crossover_point] + parents[parent1_idx][crossover_point:]
        
        offspring.append(offspring_path1)
        offspring.append(offspring_path2)
        
    offspring = np.array(offspring,dtype=object)
    return offspring

In [91]:
parents = np.array([[(0,1),(1,2),(3,3)],
                    [(2,2),(9,9),(2,2),(8,8)],
                    [(1,1),(2,2),(3,3),(4,4),(5,5)],
                    [(8,8)],
                    [(1,2),(2,3),(3,4),(4,5),(5,6),(6,7),(7,8)],
                    [(10,11),(11,12)]],dtype=object)


In [92]:
crossover(parents)

array([list([(0, 1), (1, 2), (2, 2), (8, 8)]),
       list([(2, 2), (9, 9), (3, 3)]), list([(1, 1), (2, 2)]),
       list([(8, 8), (3, 3), (4, 4), (5, 5)]),
       list([(1, 2), (2, 3), (3, 4)]),
       list([(10, 11), (11, 12), (4, 5), (5, 6), (6, 7), (7, 8)])],
      dtype=object)

## Bresenham Line Generation algorithm for Mutation operator

In [201]:
def bresenham(x0: int, y0: int, x1: int, y1: int):
    if x1-x0 == 0:
        dy = y1-y0
        sig = 1 if dy > 0 else -1
        path_list = [(x0,y0+sig*i) for i in range(sig*dy+1)]
    elif y1-y0 == 0:
        dx = x1-x0
        sig = 1 if dx > 0 else -1
        path_list = [(x0+sig*i,y0) for i in range(sig*dx+1)]
    else:
        dx = abs(x1-x0)
        sx = 1 if x0<x1 else -1
        dy = -abs(y1-y0)
        sy = 1 if y0 < y1 else -1
        err = dx+dy
        path_list = []
        x = x0
        y = y0
        while (x != x1 or y != y1):
            path_list.append( (x,y) )
            e2 = 2*err
            if (e2>=dy):
                err += dy
                x += sx
            if (e2 <= dx):
                err += dx
                y += sy

        path_list.append( (x,y) )
    
    path_list.remove( (x0,y0) )
    path_list.remove( (x1,y1) )
    return path_list

In [202]:
bresenham(0,0,10,10)

[(1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9)]

In [203]:
bresenham(-10,-10,5,0)

[(-9, -9),
 (-8, -9),
 (-7, -8),
 (-6, -7),
 (-5, -7),
 (-4, -6),
 (-3, -5),
 (-2, -5),
 (-1, -4),
 (0, -3),
 (1, -3),
 (2, -2),
 (3, -1),
 (4, -1)]

In [204]:
bresenham(0,0,1,0)

[]

In [205]:
bresenham(-5,0,5,0)

[(-4, 0), (-3, 0), (-2, 0), (-1, 0), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0)]

# Addition Mutation Operator

In [238]:
def mutation_addition(offsprings, map_shape, start, end):
    size = offsprings.shape[0]
    for i in range(size):
        path = offsprings[i]
        random_select_idx = random.randint(0,len(path)-1)
        random_select = path[random_select_idx]
        
        #getting the neighbors
        if random_select_idx == 0:
            neighbor_back = start
            neighbor_front = path[random_select_idx + 1]
        elif random_select_idx == len(path)-1:
            neighbor_back = path[random_select_idx - 1]
            neighbor_front = end
        else:
            neighbor_back = path[random_select_idx - 1]
            neighbor_front = path[random_select_idx + 1]
        
        x0,y0 = neighbor_back
        x1,y1 = random_select
        bresenham_back = bresenham(x0,y0,x1,y1)
        
        x0,y0 = random_select
        x1,y1 = neighbor_front
        bresenham_front = bresenham(x0,y0,x1,y1)
        
        len_bresenham_back = len(bresenham_back)
        len_bresenham_front = len(bresenham_front)
        
        selected_back = bresenham_back[random.randint(0,len_bresenham_back-1)] \
                        if len_bresenham_back != 0 else neighbor_back
        selected_front = bresenham_front[random.randint(0,len_bresenham_front-1)] \
                        if len_bresenham_front != 0 else neighbor_front
        
        print(f'Path: {path}')
        inserted_back = False
        if selected_back != start and selected_back != neighbor_back:
            path[random_select_idx] = selected_back
        if selected_front != end and selected_front != neighbor_front:
                path.insert(random_select_idx+1,selected_front)
        
        
        print(f'Randomized Selection: {random_select}')
        print(f'Neighbors: {neighbor_back} {neighbor_front}')
        print(f'Bresenham Back {bresenham_back}')
        print(f'Bresenham Front {bresenham_front}')
        print(f'Selected Points: {selected_back} {selected_front}')
        print(f'New Path: {path}')
        print('\n')

In [242]:
offsprings = crossover(parents)
mutation_addition(offsprings, (12,12), (0,0), (11,11))

Path: [(0, 1), (1, 2), (2, 2), (8, 8)]
Randomized Selection: (1, 2)
Neighbors: (0, 1) (2, 2)
Bresenham Back []
Bresenham Front []
Selected Points: (0, 1) (2, 2)
New Path: [(0, 1), (1, 2), (2, 2), (8, 8)]


Path: [(2, 2), (9, 9), (3, 3)]
Randomized Selection: (9, 9)
Neighbors: (2, 2) (3, 3)
Bresenham Back [(3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8)]
Bresenham Front [(8, 8), (7, 7), (6, 6), (5, 5), (4, 4)]
Selected Points: (3, 3) (7, 7)
New Path: [(2, 2), (3, 3), (7, 7), (3, 3)]


Path: [(1, 1), (2, 2)]
Randomized Selection: (1, 1)
Neighbors: (0, 0) (2, 2)
Bresenham Back []
Bresenham Front []
Selected Points: (0, 0) (2, 2)
New Path: [(1, 1), (2, 2)]


Path: [(8, 8), (3, 3), (4, 4), (5, 5)]
Randomized Selection: (5, 5)
Neighbors: (4, 4) (11, 11)
Bresenham Back []
Bresenham Front [(6, 6), (7, 7), (8, 8), (9, 9), (10, 10)]
Selected Points: (4, 4) (7, 7)
New Path: [(8, 8), (3, 3), (4, 4), (5, 5), (7, 7)]


Path: [(1, 2), (2, 3), (3, 4)]
Randomized Selection: (3, 4)
Neighbors: (2, 3) (11,

In [244]:
offsprings #updated from mutation

array([list([(0, 1), (1, 2), (2, 2), (8, 8)]),
       list([(2, 2), (3, 3), (7, 7), (3, 3)]), list([(1, 1), (2, 2)]),
       list([(8, 8), (3, 3), (4, 4), (5, 5), (7, 7)]),
       list([(1, 2), (2, 3), (3, 4), (5, 6)]),
       list([(10, 11), (11, 12), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9)])],
      dtype=object)